# Introduction

That's a strange name for a topic - "defensive programming" - defence against what? Well, bugs of course. Experience has shown that in most projects the majority of development time is spent debugging. For this reason, it is highly desirable to follow practices that minimise the number of bugs. In this notebook I will introduce a number of tried and true methods for avoiding bugs. It does require extra work, but your future self will thank you.

This notebook is by no means an exhaustive review of all good practices, but it's a good place to start. The idea here is to introduce some important concepts, about which you can read more later, and adapt to your particular application.

# Documentation

If you are working with other people, or intend for your code to be used by other people, then it goes without saying that you must document your code. However, I argue that you need to document your code even if you are the only one using it. The reason is that your future self will not remember how to use the code. 

As a rule of thumb, when it comes to documentation, more is better. The bare minimum for a documentation for a function is
1. A short description
2. Description of the arguments
3. Description of the output

Python [docstring](https://www.python.org/dev/peps/pep-0257/) feature makes the documentation available also via the interactive shell (see example below)

In [2]:
def is_even(num):
    
    """Checks if a number is even
    
    Parameters:
    num - An integer
        
    Returns:
    True if num is even, False otherwise    
    """
    
    return num%2==0

[is_even(4), is_even(3)]

[True, False]

The documentation can be shown using the ```help``` function

In [3]:
help(is_even)

Help on function is_even in module __main__:

is_even(num)
    Checks if a number is even
    
    Parameters:
    num - An integer
        
    Returns:
    True if num is even, False otherwise



I'd like to end with one word of caution. The code is alive while the documentation is not. Be sure to change the documentation when you change the code.

### Exercise

Write a function that accepts the radius of a circle and calculates the area. Document the function and use help to bring up the documentation.

# Test Driven Development

Most people, if they test their code at all, write their tests after they've written the code. In this section I will make the case for writing the tests **before** you write the code. This approach is called test driven development. Besides catching bugs early on, the benefit for this approach is that it forces you to think about the interface, while most developers would be preoccupied with the implementation. Below is a demonstration of this approach, using the [unittest](https://docs.python.org/3/library/unittest.html) package.

Suppose we want to write a function that checks if an integer divides three. We begin with the declaration of a function that does not do anything, and a test that fails

In [10]:
import unittest

def is_div_three(num):
    
    pass

class TestIsDivThree(unittest.TestCase):

    def test_3(self):
        self.assertEqual(is_div_three(3),True)
    def test_4(self):
        self.assertEqual(is_div_three(4),False)

unittest.main(argv=[''], verbosity=2, exit=False)

test_3 (__main__.TestIsDivThree) ... FAIL
test_4 (__main__.TestIsDivThree) ... FAIL

FAIL: test_3 (__main__.TestIsDivThree)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-10-e1e85a88bf75>", line 10, in test_3
    self.assertEqual(is_div_three(3),True)
AssertionError: None != True

FAIL: test_4 (__main__.TestIsDivThree)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-10-e1e85a88bf75>", line 12, in test_4
    self.assertEqual(is_div_three(4),False)
AssertionError: None != False

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (failures=2)


The next phase is to implement the function. After this stage, hopefully all the tests pass. If not, then the function or the test need to be fixed.

In [11]:
import unittest

def is_div_three(num):
    
    return num%3 == 0

class TestIsDivThree(unittest.TestCase):

    def test_3(self):
        self.assertEqual(is_div_three(3),True)
    def test_4(self):
        self.assertEqual(is_div_three(4),False)

unittest.main(argv=[''], verbosity=2, exit=False)

test_3 (__main__.TestIsDivThree) ... ok
test_4 (__main__.TestIsDivThree) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


Another good rule of thumb for writing tests is that once a bug was found and fixed, one should write a test that makes sure that the  bug does not recur.

### Exercise

Test and write a function that accepts the radius of a sphere and calculates its volume

# Exceptions

One way to spot errors early on is by checking if the data is valid. This can be done using assertions

In [6]:
def div_by_four(num):
    
    assert(type(num)==type(1))
    
    return num%4==0

print(div_by_four(4))
print(div_by_four(1))
print(div_by_four('four'))

True
False


AssertionError: 

Python's default response to errors is to exit. However, sometimes we'd like to do something else. For example, we'd like the program to give us more information when there's error. Not only that, but we'd like information outside the scope of the function where the error occurred. This can be accomplished by try catch clauses

In [12]:
def take_sqrt(num):
    
    assert(num>=0)
    
    return num**0.5

# Suppose we get the following input from the user
user_input = [1,2,3,4,5,6,-7,8,9]

# We proceed to calculate the square root from all entries
for index, num in enumerate(user_input):
    
    try:
        print(take_sqrt(num))
    except AssertionError:
        print('Something wrong with entry at position '+str(index)+'. The value there is '+str(num)+'.')

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
Something wrong with entry at position 6. The value there is -7.
2.8284271247461903
3.0


# Lint

After you've already written your code, you can have another program go over the source code and try to find errors. This sort of program is called a static code analyser, or lint. We'll use the most popular one for python, called [pylint](https://www.pylint.org/). In the following example I'll show an issue that the python interpreter doesn't catch, but that pylint does

In [19]:
with open('./bad_compare.py') as f:
    raw_text = f.read()
print(raw_text)

def bad_compare(lhs, rhs):

    unnecessary = lhs - rhs

    return rhs == lhs

print(bad_compare(3, 3))



In [20]:
!python ./bad_compare.py

True


In [21]:
!pylint bad_compare.py

************* Module bad_compare
bad_compare.py:1:0: C0114: Missing module docstring (missing-module-docstring)
bad_compare.py:1:0: C0116: Missing function or method docstring (missing-function-docstring)
bad_compare.py:3:4: W0612: Unused variable 'unnecessary' (unused-variable)

-----------------------------------

Your code has been rated at 2.50/10





In this example, the code works just find, but pylint picks up on the fact that there is an unused variables

# Logging

All the discussion above assumes everything is working. But what do we do when it doesn't? The simplest way to diagnose the problem is by inserting ```print``` statements. Unfortunately, not only is this a messy way to go about it, after you fix the problem you need to scan the code and remove all the print statements. Luckily, there is a better way - logging. Logging allows you to control the amout of output you get from a function. Typically, you'd like to minimise the output in production mode, and make it verbose when trying to find a problem. Logging let's you do just that.

In [13]:
import logging

def calc_fibo(n):
    
    if n<3:
        return 1
        
    last_term = 1
    before_last = 1
    for i in range(2,n+1):
        logging.info(str(last_term))
        next_term = last_term + before_last
        before_last = last_term
        last_term = next_term
    return last_term

Logging off

In [14]:
logging.getLogger().setLevel(logging.WARNING)
calc_fibo(5)

8

Logging on

In [15]:
logging.getLogger().setLevel(logging.INFO)
print(calc_fibo(5))
logging.getLogger().setLevel(logging.WARNING)

INFO:root:1
INFO:root:2
INFO:root:3
INFO:root:5


8


The logging library can let you do more sophisticated things like printing out the time for each instruction, or write output to a file. See the documentation.

# Debugging

Debugging is the last resort you turn to when all else fails. It lets you examine the program while it is running, but it is excruciating to use. The default debugger for python is pdb, or ipdb to debug code in notebooks. The most common controls are:

* n - Next instruction
* s - Step into function
* b - Set breakpoint
* c - Continue until breakpoint
* q - quit

In [64]:
!pip install ipdb

  Created wheel for ipdb: filename=ipdb-0.13.2-cp37-none-any.whl size=10527 sha256=38330800c75e6e1215df81b3a1b8825b63697053e4b44879b68caab9e1cc5379
  Stored in directory: C:\Users\almog\AppData\Local\pip\Cache\wheels\60\c2\15\793365e3c9318c46ba914263740d90f1fe67f544b979141ce4
Successfully built ipdb


In [65]:
import ipdb
ipdb.set_trace()

fibo_array = []
for n in range(5):
    fibo_array.append(calc_fibo(n))

--Return--
None
> <ipython-input-65-9a7dcf5d2162>(2)<module>()
      1 import ipdb
----> 2 ipdb.set_trace()
      3 

ipdb> n
> c:\users\almog\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(3334)run_code()
   3333                 # Reset our crash handler in place
-> 3334                 sys.excepthook = old_excepthook
   3335         except SystemExit as e:

ipdb> n
> c:\users\almog\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(3350)run_code()
   3349         else:
-> 3350             outflag = False
   3351         return outflag

ipdb> n
> c:\users\almog\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(3351)run_code()
   3350             outflag = False
-> 3351         return outflag
   3352 

ipdb> n
Internal StopIteration: False
> c:\users\almog\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(3254)run_ast_nodes()
   3253                         asy = compare(code)
-> 3254                     if (await self.run_code(code, result

BdbQuit: 

# Exercise

Below you will find a simple implementation of the game [Bulls and Cows](https://en.wikipedia.org/wiki/Bulls_and_Cows). Use debugging to "hack" the game and uncover the secret sequence.

In [30]:
def play_bulls_and_cows():

    import numpy

    secret = numpy.random.permutation(range(10))[:4]

    print('welcome to Bullseye')

    game_on = True
    while game_on:
        raw_guess = str(input('Enter guess (4 digits, e.g. 1234)\n'))
        print('raw guess '+raw_guess)
        assert(len(raw_guess)==4)
        guess = [int(digit) for digit in raw_guess]
        assert(len(set(guess))==4)

        # Grading
        bulleyes = 0
        hits = 0
        for n, digit in enumerate(guess):
            if digit == secret[n]:
                bulleyes += 1
            elif digit in secret:
                hits += 1
        if bulleyes == 4:
            print('You win!\n')
            game_on = False
        else:
            print(str(bulleyes)+' bulleyes and '+str(hits)+' hits\n')

# Version Control

Suppose you have a library that you tested and it is working. One day, you decide to refactor it. Maybe you want to optimise it and make it faster, or add more features. The problem is that whenever you change the code you are running the risk of breaking it. The best tool against this is version control. Version control lets you save multiple versions of your code, and load a previous version in case something breaks. It also lets you [collaborate more effectively](https://www.atlassian.com/git/tutorials/comparing-workflows). The most popular version control system today is [git](https://git-scm.com/doc?fbclid=IwAR1YxT3x6XCvpcLC1x4HCS7saF5hPAKkfr4t_IMlkk7tu9xN7FwDmgF4TBY), and the most popular repository hosting services are github and bitbucket. Below are the most commonly used commands:
* clone - Creates a copy of the repository on your local machine
* pull - Changes your local copy such that it would be the same as the one on the server
* commit - Records local changes locally
* push - Transmit commits to server

In addition, git also provides another set of commands that is especially useful for refactoring. The basic problem here is that between the current state of the code and the state of the code after refactoring, the code might be broken. This can be a problem, since you might also want to use the code in the mean while. The solution is to make two copies of the code, one that is operational, and another which is safe to break. This is called branching. After you are done making changes on the second copy and you are satisfied by the result, you can merge back the two copies of the code. The corresponding command are adequately called "branch" and "merge". The command "checkout" lets you switch between branches.

# Continuous Integration

Ideally, you'd like to test the code every time you make a change. However, doing this manually would be extremely tedious. Luckily for us, we can get a computer to do it for us. This practice is called continuous integration. The way it works is that you have a server that listens to the repository, and run a series of tests whenever a commit is pushed. One of the more popular continuous integration tools is Travis CI. Setting up continuous integration would take too long for this tutorial, but you can see an example that I've already set up [here](https://travis-ci.org/bolverk/huji-rich). If nothing else, continuous integration can help you when you argue with your colleagues about who broke the code, as seen in the example below.

<img src='travis_demo.jpg'>

# Principles of Design

## DRY - Don't repeat yourself

The two functions below do the same thing, but have different designs. Is one design better than the other? Why?

In [31]:
def trigo_galore_a(x,y):
    
    import math
    
    return [math.cos(x+y),
            math.cos(x-y),
            math.sin(x+y),
            math.sin(x-y),
            math.tan(x+y),
            math.tan(x-y)]

def trigo_galore_b(x,y):
    
    import math
    
    res = []
    for func1 in [math.cos, math.sin, math.tan]:
        for func2 in [lambda a,b:a+b, lambda a,b:a-b]:
            res.append(func1(func2(x,y)))
    return res

[trigo_galore_a(0.1, 0.2), trigo_galore_b(0.1, 0.2)]

[[0.955336489125606,
  0.9950041652780258,
  0.2955202066613396,
  -0.09983341664682815,
  0.3093362496096233,
  -0.10033467208545055],
 [0.955336489125606,
  0.9950041652780258,
  0.2955202066613396,
  -0.09983341664682815,
  0.3093362496096233,
  -0.10033467208545055]]

## Encapsulation

Which is the better way of storing info? Consider the following inventory list

In [36]:
# First option
item_names = ['invisibility cloak', 'magic potion', 'excalibur', 'winged sandals']
item_amount = [4,3,1,0]

# Second option
inventory = [('invisibility cloak', 4),
             ('magic potion', 3),
             ('excalibur', 1),
             ('winged sandals', 0)]

## Polymorphism

In the next example we consider two ways to implement a calculator function, i.e. one that accepts arguments and a type of arithmetic operation and returns the result

In [41]:
def calculator_eval_a(x, y, op):
    
    if op=='+':
        return x+y
    if op=='-':
        return x-y
    if op=='*':
        return x*y
    if op=='/':
        return x/y
    raise('Unknown operation')
    
def calculator_eval_b(x, y, op):
    
    func_dict = {'+': lambda a,b:a+b,
                 '-': lambda a,b:a-b,
                 '*': lambda a,b:a*b,
                 '/': lambda a,b:a/b}
    return func_dict[op](x,y)

[calculator_eval_a(2,3,'+'), calculator_eval_b(2,3,'+')]

[5, 5]

## Object Oriented Design

Suppose we want to write a computerised version of Dungeons and Dragons

In [62]:
import numpy

class Warrior:
    
    def __init__(self, name):
        
        self.name_ = name
        hp = sum(numpy.random.randint(1,6,size=8))
        self.max_hp_ = hp
        self.current_hp_ = hp
    
    actions = {'Slash':lambda: numpy.random.randint(1,10), 
               'Stab':lambda: numpy.random.randint(3,7)}
    
class Mage:
    
    def __init__(self, name):
        
        self.name_ = name
        hp = sum(numpy.random.randint(1,6,size=4))
        self.max_hp_ = hp
        self.current_hp_ = hp
        
    actions = {'Fireball': lambda: numpy.random.randint(1,20)}
    
class Cleric:
    
    def __init__(self, name):
        
        self.name_ = name
        hp = sum(numpy.random.randint(1,6,size=6))
        self.max_hp_ = hp
        self.current_hp_ = hp
        
    actions = {'Heal':lambda:numpy.random.randint(-10,-1)}
    
def do_action(source, action_name, target, announce=False):
    
    if announce:
        print(source.name_+' uses '+action_name+' on '+target.name_)
    
    hp_change = source.actions[action_name]()
    target.current_hp_ -= hp_change
    
caramon = Warrior('Caramon Majere')
raistlin = Mage('Raistlin Majere')
crysania = Cleric('Crysania Tarinius')

print(caramon.name_+' hp = '+str(caramon.current_hp_))
do_action(raistlin, 'Fireball', caramon, announce=True)
print(caramon.name_+' hp = '+str(caramon.current_hp_))
do_action(crysania, 'Heal', caramon, announce=True)
print(caramon.name_+' hp = '+str(caramon.current_hp_))

Caramon Majere hp = 14
Raistlin Majere uses Fireball on Caramon Majere
Caramon Majere hp = 11
Crysania Tarinius uses Heal on Caramon Majere
Caramon Majere hp = 17


# Projects

## Axelrod's Tournament

In the 70s the political scientist Robert Axelrod ran a virtual [tournament](https://en.wikipedia.org/wiki/The_Evolution_of_Cooperation#Axelrod's_tournaments). Each round, two bots would play a few rounds of the [prisoner's dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma). He solicited bot strategies from about 200 experts in related fields to determine the optimal strategy.

In this exercise you will recreate the experiment in python. Each round each bot makes a secret decision, to cooperate or defect. If both coopereate, they both score 3 points. If both defect, both score 1 point. If one defects while the other cooperates, then the former gets 5 points while the latter gets zero points. 

Create an infrastructure to pit different strategies against each other. Implement these strategies and compare their performances. Have each round last 20 turns.
Always defect - Always choose defect
Always cooperate - Always choose to cooperate
Random - Choose at random
Resentful - Start by cooperating, but if the other player defects then choose defect for the remainder of the match
Tit for Tat - Start by cooperating, continue by playing the other player's previous choice

See [here](https://ncase.me/trust/) for an interactive demo.

## Initial Value Problem

Create a framework for solving first order ODEs. Implement both [Euler's method](https://en.wikipedia.org/wiki/Euler_method) and the [Runge Kutta method](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods). Use both methods to solve both differential equations

$$\frac{d y}{d x} = \sin \left(x\right) y$$

$$\frac{d y}{d x} = - \frac{y}{y^2+0.1}$$

with initial conditions $y \left(0\right) = 1$. Integrate to $x=1$ with step size $\Delta x = 10^{-3}$.

## Coin Superposition Game

Imagine there's a machine with four compartments arranged on the vertices of a square. Each one contains a coin, which is either heads or tails. Each turn, you can choose to compartments that you can open, and then you can choose whether you want to flip none, one or both coins. After that, you close the compartments, and the machine spins fast (so you can't follow it) and stops at some random orientation. The object of the game is to have all coins facing the same way, starting from some random configuration. See this [video](https://www.youtube.com/watch?v=WcA-1QOHeeA) for more info and a solution.

Implement this game in python, where the user issues instructions through a text interface.